# KPI Inference
This notebook takes in the relevant paragraphs to KPIs found in the relevance infer stage, the fine tuned KPI EXTRACTION model from the training stage, and performs inference to return specific answers to the KPIs.

In [1]:
from config_qa_farm_train import QAFileConfig, QAInferConfig
import pprint
import pathlib
import os
from src.data.s3_communication import S3Communication
from src.models.text_kpi_infer import TextKPIInfer
from dotenv import load_dotenv
import zipfile
import config

11/09/2021 16:35:58 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("S3_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("S3_SECRET_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

In [4]:
#Settings data files and checkpoints parameters
file_config = QAFileConfig("infer_demo") 
infer_config = QAInferConfig("infer_demo")

In [ ]:
# When running in Automation using Elyra and Kubeflow Pipelines,
# set AUTOMATION = 1 as an environment variable
if os.getenv("AUTOMATION"):
        
    # inference results dir
    if not os.path.exists(infer_config.relevance_dir['Text']):
        pathlib.Path(infer_config.relevance_dir['Text']).mkdir(parents=True, exist_ok=True)

    # kpi inference results dir
    if not os.path.exists(infer_config.result_dir['Text']):
        pathlib.Path(infer_config.result_dir['Text']).mkdir(parents=True, exist_ok=True)

    # load dir
    if not os.path.exists(infer_config.load_dir['Text']):
        pathlib.Path(infer_config.load_dir['Text']).mkdir(parents=True, exist_ok=True)

    # download relevance predictions from s3 
    s3c.download_files_in_prefix_to_dir(
    config.BASE_INFER_RELEVANCE_S3_PREFIX,
    infer_config.relevance_dir['Text'],
    )

In [5]:
model_root = pathlib.Path(file_config.saved_models_dir).parent
model_rel_zip = pathlib.Path(model_root, 'KPI_EXTRACTION.zip')
s3c.download_file_from_s3(model_rel_zip, "corpdata/saved_models", "KPI_EXTRACTION.zip")
with zipfile.ZipFile(pathlib.Path(model_root, 'KPI_EXTRACTION.zip'), 'r') as z:
    z.extractall(model_root)

## Inference

We can use the saved model and test it on some real examples.<br><br>
First let's load the model:

In [6]:
file_config.saved_models_dir

'/opt/app-root/src/aicoe-osc-demo/models/KPI_EXTRACTION'

In [7]:
tki = TextKPIInfer(infer_config)

11/09/2021 16:37:20 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
11/09/2021 16:37:23 - WARNING - farm.infer -   QAInferencer always has task_type='question_answering' even if another value is provided to Inferencer.load() or QAInferencer()


Now, let's make prediction on a pair of paragraph and question.

In [8]:
context = """the paris agreement on climate change drafted in 2015 aims to reduce worldwide emissions of greenhouse 
gases to a level intended to limit a rise in global temperatures to below 2 degrees or, better still,
to below 1.5 degrees. verbund’s target of reducing greenhouse gas emissions by 90% measured beginning from 
the basis year 2011 5 million tonnes co2e until 2021 includes scope 1, scope 2 market- based and parts of scope 3 emissions 
for energy and air travel. the science based targets initiative validated this goal as science-based in october 2016, 
i.e. it meets global standards. according to current planning, the target can be achieved. 
however, if the grid operator requires higher generation volumes 
"""
question = "What is the target year for climate commitment?"
    

In [9]:
QA_input = [
        {
            "qas": [question],
            "context":  context
        }]

result = tki.infer_on_dict(QA_input)[0]
pprint.pprint(result)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.79 Batches/s]

{'predictions': [{'answers': [{'answer': '2021',
                               'context': 'the basis year 2011 5 million '
                                          'tonnes co2e until 2021 includes '
                                          'scope 1, scope 2 market- based and '
                                          'par',
                               'document_id': '0-0',
                               'offset_answer_end': 366,
                               'offset_answer_start': 362,
                               'offset_context_end': 414,
                               'offset_context_start': 314,
                               'probability': None,
                               'score': 7.129119873046875},
                              {'answer': 'no_answer',
                               'context': '',
                               'document_id': '0-0',
                               'offset_answer_end': 0,
                               'offset_answer_start': 0,
      

What does the prediction result show? 

In [10]:
# This is the best answer. Generally it can be span-based or it can be no-answer, which ever is higher
# Here the top answer is the span '2021'
result['predictions'][0]['answers'][0]

{'score': 7.129119873046875,
 'probability': None,
 'answer': '2021',
 'offset_answer_start': 362,
 'offset_answer_end': 366,
 'context': 'the basis year 2011 5 million tonnes co2e until 2021 includes scope 1, scope 2 market- based and par',
 'offset_context_start': 314,
 'offset_context_end': 414,
 'document_id': '0-0'}

In [11]:
# Non-answerable score: The model is pretty confident that the answer to the question can be in the context.
result['predictions'][0]['answers'][1]

{'score': -20.135550498962402,
 'probability': None,
 'answer': 'no_answer',
 'offset_answer_start': 0,
 'offset_answer_end': 0,
 'context': '',
 'offset_context_start': 0,
 'offset_context_end': 0,
 'document_id': '0-0'}

Now, let's use the model to infer kpi answers from the relevance results 

In [12]:
infer_config.relevance_dir

{'Text': '/opt/app-root/src/aicoe-osc-demo/data/infer_relevance'}

In [13]:
tki.infer_on_relevance_results(infer_config.relevance_dir['Text'])

11/09/2021 16:37:24 - INFO - src.models.text_kpi_infer -   #################### Starting KPI Inference for the following relevance CSV files found in /opt/app-root/src/aicoe-osc-demo/data/infer_kpi:
['sustainability-report-2019_predictions_relevant.csv'] 
11/09/2021 16:37:24 - INFO - src.models.text_kpi_infer -   #################### 1/1
11/09/2021 16:37:24 - INFO - src.models.text_kpi_infer -   Starting KPI Extraction for sustainability-report-2019
Inferencing Samples: 100%|██████████| 45/45 [01:08<00:00,  1.53s/ Batches]
11/09/2021 16:38:35 - ERROR - farm.modeling.predictions -   Both start and end offsets should be 0: 
226, 226 with a no_answer. 
11/09/2021 16:38:35 - INFO - src.models.text_kpi_infer -   Save the result of KPI extraction to /opt/app-root/src/aicoe-osc-demo/data/infer_kpi/sustainability-report-2019_predictions_kpi.csv


,pdf_name,kpi,kpi_id,answer,page,paragraph,source,score,no_ans_score,no_answer_score_plus_boost,index
0,sustainability-report-2019,What is the company name?,NaN,Equinor ASA,32.0,Equinor ASA Box 8500 NO-4035 Stavanger Norway ...,Text,16.310862,-10.405624,-25.405624,NaN
1,sustainability-report-2019,What is the company name?,NaN,Equinor,27.0,To show our commitment to equal and inclusive ...,Text,16.253649,-7.369206,-22.369206,NaN
2,sustainability-report-2019,What is the company name?,NaN,Equinor,29.0,Payments made directly by Equinor to governmen...,Text,16.137005,-8.840895,-23.840895,NaN
3,sustainability-report-2019,What is the company name?,NaN,Equinor Sustainability,20.0,Equinor Sustainability report 2019 Always safe...,Text,15.984804,-5.345298,-20.345298,NaN


In [14]:
# upload the predicted files to s3
s3c.upload_files_in_dir_to_prefix(
    infer_config.result_dir['Text'],
    config.BASE_INFER_KPI_S3_PREFIX
)

# Conclusion
This notebook ran the _KPI_ inference on a sample dataset and stored the output in a csv format.